In [1]:
%pylab inline
import pandas as pd

from dataManager import DataManager
from model import ModelManager as manager

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse

import plotly.graph_objects as go

Populating the interactive namespace from numpy and matplotlib


In [2]:
sales = DataManager().sales_prod.copy().query('VIGENCIA != "DESCONTINUADO"')
sales['AREA'] = sales['ANCHO']*sales['FONDO']
sales = sales.groupby(['REF','TIENDA']).agg({'PRECIO':'mean','DESCUENTO(%)':'mean','AREA':'first',
                                            'ALTO':'first','PUESTOS':'first', 'COLOR_POS':'first', 
                                            'SUBCATEGORIA_POS':'first','MATERIAL_POS':'first','ACABADO':'first',
                                            'CATEGORIA':'first','ORIGEN':'first'}).reset_index()

# 2021 future months and covid
months=[5,6,7,8,9,10,11,12]
covid=[1,1,1,1,1,1,1,1]
aux0=sales[['REF','TIENDA']].copy()
for m,c in zip(months,covid):
    aux0[m]=c

aux1=pd.melt(aux0,id_vars=['REF','TIENDA'],var_name='MES',value_name='F_COVID')

final_df_future=aux1.merge(sales,on=['REF','TIENDA'],how='left',validate='m:1')

final_df_future['ANIO'] = 2021
final_df_future['DATE'] = final_df_future['ANIO'].astype(str) + '-' +final_df_future['MES'].astype(str).str.zfill(2)
final_df_future.sort_values(['ANIO','MES']).reset_index(drop=True)

futuro = final_df_future[['REF','TIENDA','DATE','ANIO','MES','PRECIO','DESCUENTO(%)','F_COVID','AREA','ALTO','PUESTOS','COLOR_POS','SUBCATEGORIA_POS','MATERIAL_POS','ACABADO','ORIGEN']]
futuro[(futuro.REF == 'BR0002:00193:') & (futuro.TIENDA == 'PAGINA WEB FIOTTI')]
futuro

,REF,TIENDA,DATE,ANIO,MES,PRECIO,DESCUENTO(%),F_COVID,AREA,ALTO,PUESTOS,COLOR_POS,SUBCATEGORIA_POS,MATERIAL_POS,ACABADO,ORIGEN
0,BR0002:00193:,PAGINA WEB FIOTTI,2021-05,2021,5,3.320816e+05,0.529850,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO
1,BR0002:00193:,PUNTO DE VENTA AV 68,2021-05,2021,5,3.221593e+05,0.453105,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO
2,BR0002:00193:,PUNTO DE VENTA CALLE 134,2021-05,2021,5,3.241880e+05,0.392612,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO
3,BR0002:00193:,PUNTO DE VENTA CENTRO MAYOR,2021-05,2021,5,3.241880e+05,0.392612,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO
4,BR0002:00193:,PUNTO DE VENTA PLAZA CENTRAL,2021-05,2021,5,3.284670e+05,0.468105,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28035,XZ0987:00024:,PUNTO DE VENTA SOACHA,2021-12,2021,12,1.006722e+06,0.600000,1,10500.0,75.0,6.0,ROJO,SET 6P,VIDRIO,BRILLANTE,IMPORTADO
28036,XZ0989:00024:,ADMINISTRACION,2021-12,2021,12,8.021830e+05,0.599999,1,8400.0,75.0,4.0,ROJO,SET 4P,METAL + VIDRIO,MATE,IMPORTADO
28037,XZ0989:00024:,PAGINA WEB FIOTTI,2021-12,2021,12,4.392000e+05,0.000000,1,8400.0,75.0,4.0,ROJO,SET 4P,METAL + VIDRIO,MATE,IMPORTADO
28038,XZ0989:00024:,PUNTO DE VENTA CALI,2021-12,2021,12,6.709250e+05,0.500001,1,8400.0,75.0,4.0,ROJO,SET 4P,METAL + VIDRIO,MATE,IMPORTADO


In [3]:
pasado = DataManager().sales_ref_month_sin_ventas_mayores().copy().query('VIGENCIA != "DESCONTINUADO"')
pasado

,ANIO,MES,REF,TIENDA,PRECIO,SUBTOTAL,DESCUENTO(%),TOTAL,CANTIDAD,ALTO,...,COLOR_POS,CATEGORIA,SUBCATEGORIA_POS,VIGENCIA,ORIGEN,ESTILO,MATERIAL_POS,ACABADO,F_COVID,DATE
0,2019,1,D00510:00079:00079,PAGINA WEB FIOTTI,660402.0,660402,0.000000,660402,1,90.0,...,NEGRO,SALAS Y SOFAS,OTROS,LINEA,MANUFACTURADO,MODERNO,otros,TEXTILES SUAVES,0,2019-01
1,2019,1,D00510:00079:00079,PUNTO DE VENTA AV 68,854416.0,854416,0.410670,503533,1,90.0,...,NEGRO,SALAS Y SOFAS,OTROS,LINEA,MANUFACTURADO,MODERNO,otros,TEXTILES SUAVES,0,2019-01
2,2019,1,D00510:00079:00079,PUNTO DE VENTA CALLE 134,854416.0,2563248,0.410670,1510599,3,90.0,...,NEGRO,SALAS Y SOFAS,OTROS,LINEA,MANUFACTURADO,MODERNO,otros,TEXTILES SUAVES,0,2019-01
3,2019,1,D00510:00080:00080,PUNTO DE VENTA CENTRO MAYOR,854416.0,854416,0.410670,503533,1,90.0,...,BLANCO,SALAS Y SOFAS,OTROS,LINEA,MANUFACTURADO,MODERNO,otros,TEXTILES SUAVES,0,2019-01
4,2019,1,D00510:00112:00112,PUNTO DE VENTA CALLE 134,1050933.0,1050933,0.500000,525466,1,90.0,...,CAFE,SALAS Y SOFAS,OTROS,LINEA,MANUFACTURADO,MODERNO,otros,TEXTILES SUAVES,0,2019-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20457,2021,4,XZ0573:00005:,VILLA DEL RIO,700016.0,700016,0.391227,426151,1,76.0,...,NEGRO,COMEDORES,MESAS DE 4P,LINEA,IMPORTADO,MODERNO,VIDRIO,BRILLANTE,1,2021-04
20463,2021,4,XZ0737:00001:,PUNTO DE VENTA CALLE 134,1359198.0,1359198,0.390694,828168,1,75.0,...,BLANCO,COMEDORES,SET 6P,LINEA,IMPORTADO,MODERNO,VIDRIO,BRILLANTE,1,2021-04
20466,2021,4,XZ0897:00044:,PUNTO DE VENTA AV 68,1160663.0,1160663,0.390728,707159,1,75.0,...,OTRO,COMEDORES,OTROS,LINEA,IMPORTADO,MODERNO,VIDRIO,BRILLANTE,1,2021-04
20467,2021,4,XZ0897:00044:,PUNTO DE VENTA POBLADO,1088256.0,1088256,0.400000,652954,1,75.0,...,OTRO,COMEDORES,OTROS,LINEA,IMPORTADO,MODERNO,VIDRIO,BRILLANTE,1,2021-04


In [4]:
scaler = MinMaxScaler()

num_var=['AREA','ALTO','DESCUENTO(%)','PRECIO','CANTIDAD']
x_num=pasado[num_var[:-1]].astype('float')
x_num_norm = scaler.fit_transform(x_num)

cat_var=[
    'TIENDA','MES',
    'F_COVID','PUESTOS','COLOR_POS','SUBCATEGORIA_POS',
    'MATERIAL_POS','ACABADO','ORIGEN'
]
x_cat=pasado[cat_var].astype('category')
x_cat_dummies=pd.get_dummies(x_cat)

x = np.append(x_num_norm,x_cat_dummies,axis=1)
y = pasado['CANTIDAD']

model = GradientBoostingRegressor(**{'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200})
model.fit(x,y)

GradientBoostingRegressor(learning_rate=0.01, max_depth=6, n_estimators=200)

In [12]:
total = pd.concat([pasado.drop(columns='CANTIDAD').copy(),futuro.copy()]).reset_index(drop=True)
particion = 10275
total[particion:]

,ANIO,MES,REF,TIENDA,PRECIO,SUBTOTAL,DESCUENTO(%),TOTAL,ALTO,AREA,...,COLOR_POS,CATEGORIA,SUBCATEGORIA_POS,VIGENCIA,ORIGEN,ESTILO,MATERIAL_POS,ACABADO,F_COVID,DATE
10275,2021,5,BR0002:00193:,PAGINA WEB FIOTTI,3.320816e+05,NaN,0.529850,NaN,180.0,2583.0,...,OTRO,NaN,OTROS,NaN,IMPORTADO,NaN,MADERA LAMINADA,MATE,1,2021-05
10276,2021,5,BR0002:00193:,PUNTO DE VENTA AV 68,3.221593e+05,NaN,0.453105,NaN,180.0,2583.0,...,OTRO,NaN,OTROS,NaN,IMPORTADO,NaN,MADERA LAMINADA,MATE,1,2021-05
10277,2021,5,BR0002:00193:,PUNTO DE VENTA CALLE 134,3.241880e+05,NaN,0.392612,NaN,180.0,2583.0,...,OTRO,NaN,OTROS,NaN,IMPORTADO,NaN,MADERA LAMINADA,MATE,1,2021-05
10278,2021,5,BR0002:00193:,PUNTO DE VENTA CENTRO MAYOR,3.241880e+05,NaN,0.392612,NaN,180.0,2583.0,...,OTRO,NaN,OTROS,NaN,IMPORTADO,NaN,MADERA LAMINADA,MATE,1,2021-05
10279,2021,5,BR0002:00193:,PUNTO DE VENTA PLAZA CENTRAL,3.284670e+05,NaN,0.468105,NaN,180.0,2583.0,...,OTRO,NaN,OTROS,NaN,IMPORTADO,NaN,MADERA LAMINADA,MATE,1,2021-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38310,2021,12,XZ0987:00024:,PUNTO DE VENTA SOACHA,1.006722e+06,NaN,0.600000,NaN,75.0,10500.0,...,ROJO,NaN,SET 6P,NaN,IMPORTADO,NaN,VIDRIO,BRILLANTE,1,2021-12
38311,2021,12,XZ0989:00024:,ADMINISTRACION,8.021830e+05,NaN,0.599999,NaN,75.0,8400.0,...,ROJO,NaN,SET 4P,NaN,IMPORTADO,NaN,METAL + VIDRIO,MATE,1,2021-12
38312,2021,12,XZ0989:00024:,PAGINA WEB FIOTTI,4.392000e+05,NaN,0.000000,NaN,75.0,8400.0,...,ROJO,NaN,SET 4P,NaN,IMPORTADO,NaN,METAL + VIDRIO,MATE,1,2021-12
38313,2021,12,XZ0989:00024:,PUNTO DE VENTA CALI,6.709250e+05,NaN,0.500001,NaN,75.0,8400.0,...,ROJO,NaN,SET 4P,NaN,IMPORTADO,NaN,METAL + VIDRIO,MATE,1,2021-12


In [13]:
num=['AREA','ALTO','DESCUENTO(%)','PRECIO','CANTIDAD']
x_num=total[num[:-1]].astype('float')
x_norm = scaler.fit_transform(x_num)

cat=[
    'TIENDA','MES',
    'F_COVID','PUESTOS','COLOR_POS','SUBCATEGORIA_POS',
    'MATERIAL_POS','ACABADO','ORIGEN'
]
x_cat=total[cat].astype('category')
x_dummies=pd.get_dummies(x_cat)

x = np.append(x_num,x_dummies,axis=1)
x = x[particion:]

y = model.predict(x)

In [14]:
futuro['PREDICTED'] = y.round()
futuro

,REF,TIENDA,DATE,ANIO,MES,PRECIO,DESCUENTO(%),F_COVID,AREA,ALTO,PUESTOS,COLOR_POS,SUBCATEGORIA_POS,MATERIAL_POS,ACABADO,ORIGEN,PREDICTED
0,BR0002:00193:,PAGINA WEB FIOTTI,2021-05,2021,5,3.320816e+05,0.529850,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO,2.0
1,BR0002:00193:,PUNTO DE VENTA AV 68,2021-05,2021,5,3.221593e+05,0.453105,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO,3.0
2,BR0002:00193:,PUNTO DE VENTA CALLE 134,2021-05,2021,5,3.241880e+05,0.392612,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO,2.0
3,BR0002:00193:,PUNTO DE VENTA CENTRO MAYOR,2021-05,2021,5,3.241880e+05,0.392612,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO,2.0
4,BR0002:00193:,PUNTO DE VENTA PLAZA CENTRAL,2021-05,2021,5,3.284670e+05,0.468105,1,2583.0,180.0,NaN,OTRO,OTROS,MADERA LAMINADA,MATE,IMPORTADO,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28035,XZ0987:00024:,PUNTO DE VENTA SOACHA,2021-12,2021,12,1.006722e+06,0.600000,1,10500.0,75.0,6.0,ROJO,SET 6P,VIDRIO,BRILLANTE,IMPORTADO,2.0
28036,XZ0989:00024:,ADMINISTRACION,2021-12,2021,12,8.021830e+05,0.599999,1,8400.0,75.0,4.0,ROJO,SET 4P,METAL + VIDRIO,MATE,IMPORTADO,2.0
28037,XZ0989:00024:,PAGINA WEB FIOTTI,2021-12,2021,12,4.392000e+05,0.000000,1,8400.0,75.0,4.0,ROJO,SET 4P,METAL + VIDRIO,MATE,IMPORTADO,5.0
28038,XZ0989:00024:,PUNTO DE VENTA CALI,2021-12,2021,12,6.709250e+05,0.500001,1,8400.0,75.0,4.0,ROJO,SET 4P,METAL + VIDRIO,MATE,IMPORTADO,2.0


In [18]:
futuro['PREDICTED_M'] = (futuro.PREDICTED / 5).round()

d_futuro = futuro.groupby(['DATE']).sum().reset_index()
d_pasado = pasado.groupby(['DATE']).sum().reset_index()
defi = d_futuro#.query('REF=="D00935:00048:00048"')
fig = go.Figure()
fig.add_scatter(x=defi['DATE'], y=defi['PREDICTED_M'], mode='lines', name='Valores predichos')
fig.add_scatter(x=d_pasado['DATE'], y=d_pasado['CANTIDAD'], mode='lines', name='Valores reales')


In [21]:
sales = DataManager().sales_prod.copy()
n = len(sales[(sales.ANIO == 2021) & (sales.MES == 4)].REF.unique())
m = len(sales.REF.unique())
print(n%m)

284


In [22]:
g = len(futuro[(futuro.ANIO == 2021) & (futuro.MES == 5) & (futuro.PREDICTED_M > 0)].REF.unique())
h = len(futuro.REF.unique())
print(g,h)

413 584
